In [1]:
import csv

# Read the file
with open('test.csv', 'r') as file:
    reader = csv.reader(file)
    first_column_data = [row[0] for row in reader]

# Write the first column back to the file
with open('test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for data in first_column_data:
        writer.writerow([data])
